# TODO: 概要

# ライブラリのインポート

In [18]:
import os
import json
import datetime

from dotenv import load_dotenv, find_dotenv

from semantic_kernel.agents import (
    ChatCompletionAgent,
    ChatHistoryAgentThread,
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
from semantic_kernel.contents import (
    FunctionCallContent,
    FunctionResultContent,
    TextContent,
)


# 環境変数の取得

In [19]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")

# ユーティリティ関数

In [20]:
async def print_thread_message_details(thread: str):
    """
    スレッドのメッセージ詳細を表示します。

    Args:
        thread (str): スレッドのインスタンス
    """
    async for message in thread.get_messages():
        print("-----")

        for item in message.items:
            if isinstance(item, FunctionCallContent):
                print(f"[Function Calling] by {message.ai_model_id}")
                print(f" - Function Name : {item.name}")
                print(f" - Arguments     : {item.arguments}")

            elif isinstance(item, FunctionResultContent):
                print(f"[Function Result]")
                # 文字列のデコード変換
                if isinstance(item.result, str):
                    try:
                        decoded = json.loads(item.result)
                        print(f" - Result        : {decoded}") # デコード成功時は変換後の値を表示
                    except json.JSONDecodeError:
                        print(f" - Result        : {item.result}")  # デコード失敗時はそのまま
                else:
                    print(f" - Result        : {item.result}")

            elif isinstance(item, TextContent):
                if message.name:
                    print(f"[Agent Response] from {message.ai_model_id}")
                else:
                    print("[User Message]")
                print(f" - Content       : {item.text}")

            else:
                print(f"[Unknown Item Type] ({type(item).__name__})")
                print(f" - Raw Item      : {item}")


def log_with_timestamp(message: str) -> None:
    """
    現在時刻付きでメッセージを標準出力にログとして表示します。

    Args:
        message (str): 出力するログメッセージ。
    """
    timestamp = datetime.datetime.now().strftime("%H:%M:%S.%f")[:-3]
    print(f"[{timestamp}] {message}")

# プラグインの作成

In [28]:
# MCPのエンドポイントURL
MCP_URL = "http://127.0.0.1:8000/mcp/"


mcp_plugin = MCPStreamableHttpPlugin(
    name="mcp_plugin",
    url=MCP_URL
)

await mcp_plugin.connect()

# クライアントの初期化

In [22]:
# Chat Completion API クライアントの初期化
azure_completion_service  = AzureChatCompletion(
    service_id="azure_completion_agent",
    deployment_name=AZURE_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

In [ ]:
# Chat Completion Agent クライアントの初期化
agent = ChatCompletionAgent(
    service=azure_completion_service,
    name="mcp_agent",
    instructions=(
        "あなたはECサイトの分析エージェントです。"
        "PostgreSQLの各種マスター、注文、ユーザー情報と、CosmosDBに格納されたレビュー分析データを、mcp_pluginとして利用できます。"
    ),
    plugins=[mcp_plugin],
)

## スレッドの作成

In [24]:
# Thread の作成
thread = ChatHistoryAgentThread()
print(f"Created Thread. THREAD_ID: {thread.id}")


Created Thread. THREAD_ID: thread_de48f5c595014f3dadf2fd831fc12b19


## レスポンスを取得

In [25]:
response = await agent.get_response(
    messages=["2024年12月は何がどれくらい購入されましたか？"],
    thread=thread,
)

print(response)

2024年12月に購入された商品とその数量は以下の通りです。

---

- Surface Pro：8台
  - 購入内訳：2台（12/9）、1台（12/31）、2台（12/25）、3台（12/28）
- Surface Laptop：1台（12/18）
- Surface Go：1台（12/3）
- Xbox Game Pass：1件（12/6）
- Office：1本（12/14）
- Modern Webcam：3台（12/5）

---

数量はいずれも12月の注文ごとの合計です。  
他に詳しく知りたい商品や、カテゴリ単位・金額集計なども可能です。ご希望があればお知らせください。


In [26]:
await print_thread_message_details(thread)

-----
[User Message]
 - Content       : 2024年12月は何がどれくらい購入されましたか？
-----
[Function Calling] by gpt-4.1
 - Function Name : mcp_plugin-get_users
 - Arguments     : {}
-----
[Function Result]
 - Result        : [TextContent(inner_content=TextContent(type='text', text='[{"user_id": 1, "user_name": "user01", "email": "user01@example.com"}, {"user_id": 2, "user_name": "user02", "email": "user02@example.com"}, {"user_id": 3, "user_name": "user03", "email": "user03@example.com"}, {"user_id": 4, "user_name": "user04", "email": "user04@example.com"}, {"user_id": 5, "user_name": "user05", "email": "user05@example.com"}, {"user_id": 6, "user_name": "user06", "email": "user06@example.com"}, {"user_id": 7, "user_name": "user07", "email": "user07@example.com"}, {"user_id": 8, "user_name": "user08", "email": "user08@example.com"}, {"user_id": 9, "user_name": "user09", "email": "user09@example.com"}, {"user_id": 10, "user_name": "user10", "email": "user10@example.com"}]', annotations=None, meta=None), a